In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re

AttributeError: ignored

In [ ]:
postings = pd.read_pickle('ml_job_descriptions.pkl')

In [ ]:
from tkinter.constants import X

Getting first two keys from descriptiopn

In [ ]:
postings = postings.loc[postings['posted'].apply(lambda x: isinstance(x, str))]

In [ ]:
#sample lambda
postings[postings['description'].apply(lambda x: len(x['criteria'])==1)]['description'].iloc[0]

Getting Criteria from description

In [ ]:
stop_words = set(stopwords.words('english'))
from collections import Counter
my_counter = Counter()
for sentence in postings['Description String']:
  sentence = sentence.lower()
  words = sentence.split()
  words = [w for w in words if not w in stop_words]
  my_counter.update(words)

In [ ]:
my_counter = dict(my_counter)

In [ ]:
sorted_count = {k: v for k, v in sorted(my_counter.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
sorted_count

In [ ]:
import spacy
from collections import Counter
from string import punctuation
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 17629670
def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN']
    doc = nlp(text.lower())
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            result.append(token.text)
    return result


In [ ]:
text = ''
for sentence in postings['Description String']:
  text += sentence
output = set(get_hotwords(text))
most_common_list = Counter(output).most_common(10)
for item in most_common_list:
  print(item[0])

JESS KEYWORD EXTRACTION STARTS HERE

In [ ]:
%%capture
!pip install transformers
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from collections import Counter
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
lemmatizer = WordNetLemmatizer()
lemma = lemmatizer.lemmatize
my_stop_words = [lemma(t) for t in stopwords.words('english')]

def remove_punctuation(text):
    table = text.maketrans({key: None for key in string.punctuation})
    text = text.translate(table)
    return text

def tokenize(text):
    no_punct = remove_punctuation(text)
    stems = [lemma(t) for t in word_tokenize(no_punct) if not t in my_stop_words ]
    return stems

def naive_terms(texts, n=3):
    print("Term finding started.")
    vectorizer = CountVectorizer(tokenizer = tokenize, strip_accents = 'ascii', ngram_range = (1,n))#, stop_words=my_stop_words)
    X = vectorizer.fit_transform(texts)
    terms = vectorizer.inverse_transform(X)
    print("Term finding finished.")
    return terms

In [ ]:
skills_texts = postings['Description String']
terms = naive_terms(skills_texts)
terms = list(x for c in terms for x in c)
my_counter = Counter()
my_counter.update(terms)
# Code to sort based in count value
#sorted_count = {k: v for k, v in sorted(my_counter.items(), key=lambda item: item[1], reverse=True)}
most_common = [ x[0] for x in my_counter.most_common(15000) ]

Term finding started.


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Term finding finished.


In [ ]:
import pickle
import numpy as np
import os
from transformers import AutoTokenizer, T5EncoderModel
import torch
import torch.nn.functional as F
from tqdm import tqdm
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

device = "cuda" if torch.cuda.is_available() else "cpu"
def embedd_bert(text):
  st_model = 't5-large'
  batch_size = 32
  tokenizer = AutoTokenizer.from_pretrained(st_model)
  model = T5EncoderModel.from_pretrained(st_model).to(device)
  word_embeddings = []
  for i in tqdm(range(0,len(text),batch_size), desc="Embedding for "+" Size: "+str(len(text))):
    encoded_input = tokenizer(text[i:i+batch_size], return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
      model_output = model(**encoded_input)
      word_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
      word_embedding = F.normalize(word_embedding, p=2, dim=1)
      word_embeddings = word_embeddings + word_embedding.tolist()
  return word_embeddings

In [ ]:
word_embeddings = embedd_bert(most_common)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Embedding for  Size: 15000: 100%|██████████| 469/469 [23:03<00:00,  2.95s/it]


In [ ]:
keyword_embeddings = pd.DataFrame({'Keyword':most_common, 'embedding':word_embeddings})

In [ ]:
from nltk.cluster import KMeansClusterer
import nltk

def clustering(data,NUM_CLUSTERS = 15):

    sentences = data['Keyword']

    X = np.array(data['embedding'].tolist())

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=25,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    data['cluster'] = pd.Series(assigned_clusters, index=data.index)
    data['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

    return data

In [ ]:
keyword_clusters = clustering(keyword_embeddings, 5)

In [ ]:
keyword_clusters[['Keyword','cluster']]

In [ ]:
new_subgroup = keyword_clusters.query('cluster==0 or cluster==2')

In [ ]:
new_subgroup_cluster = clustering(new_subgroup, 3)

<ipython-input-46-537df3b12da1>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cluster'] = pd.Series(assigned_clusters, index=data.index)
<ipython-input-46-537df3b12da1>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])


In [ ]:
new_subgroup_cluster[['Keyword','cluster']]

Location Extraction

In [ ]:
postings = pd.read_pickle('ml_job_descriptions.pkl')
states_csv = pd.read_csv('States.csv')
state_name = states_csv['state_name'].tolist()
state_abbreviation = states_csv['abbreviation'].tolist()
country = states_csv['country'].tolist()

In [ ]:
def extract_location(location_string):
  location_split = location_string.split(',')
  location_dict = {}
  while location_split:
    location = location_split.pop()
    location = location.strip()
    if location in country:
      location_dict['country'] = location
    elif location in state_abbreviation:
      location_index = state_abbreviation.index(location)
      location_dict['state'] = state_name[location_index]
    elif location in state_name:
      location_dict['state'] = location
    else:
      location_dict['location'] = location
  return location_dict

In [ ]:
location_list = []
state_list = []
country_list = []
for index, row in postings.iterrows():
  location = row['Location']
  extracted_location = extract_location(location)
  print(extracted_location)
  location_list.append(extracted_location['location'] if 'location' in extracted_location else None)
  state_list.append(extracted_location['state'] if 'state' in extracted_location else None)
  country_list.append(extracted_location['country'] if 'country' in extracted_location else None)

In [ ]:
postings['location'] = location_list
postings['state'] = state_list
postings['country'] = country_list

Extracting Posted

In [ ]:
postings[['Title','posted']]

In [ ]:
from datetime import date, timedelta
current_date = date(2023, 7, 23)
posted_day = []
for index, row in postings.iterrows():
  posted = row['posted']
  if type(posted) == str:
    posted_split = posted.split(' ')
    if posted_split[1] == 'minutes' or posted_split[1] == 'hours' or posted_split[1] == 'hour' or posted_split[1] == 'minute':
      posted_day.append(current_date)
    elif posted_split[1] == 'days' or posted_split[1] == 'day':
      posted_day.append(current_date - timedelta(days=int(posted_split[0])))
    elif posted_split[1] == 'week' or posted_split[1] == 'weeks':
      posted_day.append(current_date - timedelta(weeks=int(posted_split[0])))
    elif posted_split[1] == 'months' or posted_split[1] == 'month':
      posted_day.append(current_date - timedelta(weeks=int(posted_split[0])*4))
  else:
    posted_day.append(current_date)

In [ ]:
postings['posted_date'] = posted_day

Removing Jobs without description

In [ ]:
postings['has_description'] = postings["posted"].apply(lambda x: isinstance(x, str))
postings = postings[postings['has_description']]
postings.reset_index(drop=True, inplace=True)

Final Table for database

In [ ]:
postings = postings[['id', 'Title', 'Company', 'posted_date', 'applicants', 'Employment type', 'Seniority level', 'Job function', 'Industries', 'location', 'state', 'country', 'Description String', 'url', 'company_url']]

In [ ]:
postings.to_pickle('ml_descriptions_forsql.pkl')

In [ ]:
postings.to_json('ml_descriptions_forsql.json')

In [ ]:
postings.to_csv('ml_descriptions_forsql.csv')

In [ ]:
postings[['id','Title','url']]

,id,Title,url
0,3668730857,Data Analyst,https://ca.linkedin.com/jobs/view/data-analyst...
1,3668302825,Data Scientist,https://ca.linkedin.com/jobs/view/data-scienti...
2,3668126664,Data Scientist (Optimization),https://ca.linkedin.com/jobs/view/data-scienti...
3,3670068034,Data Scientist,https://ca.linkedin.com/jobs/view/data-scienti...
4,3670303702,Data Scientist - Toronto,https://ca.linkedin.com/jobs/view/data-scienti...
...,...,...,...
5055,3668302176,Part Time - Clinical Robot Associate - TH-HEB ...,https://www.linkedin.com/jobs/view/part-time-c...
5056,3674124701,Research Associate-Fixed Term,https://www.linkedin.com/jobs/view/research-as...
5057,3672387114,Back-End Developer (Entry Level) - US/Canada,https://www.linkedin.com/jobs/view/back-end-de...
5058,3670361250,AI/Data/Visualization Department Manager,https://www.linkedin.com/jobs/view/ai-data-vis...
